In [ ]:
# Imports
import openai
import requests
from bs4 import BeautifulSoup
import json
import datetime
import my_target_job
import my_secrets

In [ ]:
# Set OpenAI API key
openai.api_key = my_secrets.OPENAI_API_KEY

In [ ]:
# Get job description from URL

# Send a GET request to the URL
url_response = requests.get(my_target_job.job_description_url)

# Extract the HTML content from the response
html_content = url_response.text

# Create a BeautifulSoup object to parse the HTML
soup = BeautifulSoup(html_content, "html.parser")

# Extract the text from the webpage
job_description = soup.get_text()

# Remove whitespace characters
job_description = " ".join(job_description.split())  

print(job_description)

In [ ]:
# Helper function: Get ChatGPT response from text prompt using API
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature,  # degree of randomness of GPT response
    )
    return response.choices[0].message["content"]

In [ ]:
# Extract information from job description

# Delimiter used to mark the start and end of the job description
delimiter = "####"

# Prompt for extracting information from job description
extraction_prompt = f"""
Your task is to extract information about the employer, job title, requirements, tasks, 
contact person, and address from the job description marked with {delimiter} characters.

Format your answer as a Python dictionary with "employer", "job title", "requirements", 
"tasks", "contact person", and "address" as keys.

Format the "requirements" and "tasks" as lists.

If the information is missing in the job description, use "unknown" as the value.

Respond as concisely as possible.

Job description: {delimiter}{job_description}{delimiter}
"""

# Get extraction response
extraction_response_str = get_completion(extraction_prompt)

# Convert extraction response from str to json
extraction_response = json.loads(extraction_response_str)

# Show extracted information
print(extraction_response)

In [ ]:
# Create cover letter

# Cover letter prompt
cover_letter_prompt = f"""
Your task is to create a professional cover letter.

Address the letter to the following employer, address, job position, and contact person:
Employer: {extraction_response["employer"]}
Address: {extraction_response["address"]}
Job Position: {extraction_response["job title"]}
Contact Person: {extraction_response["contact person"]}

Use the following sender information:
Name: {my_secrets.name}
Address: {my_secrets.address}
Phone Number: {my_secrets.phone}
Email: {my_secrets.email}

Include the location and date in the letterhead of the cover letter.
Use the location from: {my_secrets.address}
Use the current date in the English date format: {datetime.date.today()}

Describe how the education, work experience, skills, and motivation fulfill 
the job requirements and tasks. Use the following information:
Requirements: {extraction_response["requirements"]}
Tasks: {extraction_response["tasks"]}
Education: {my_secrets.education}
Work Experience: {my_secrets.work_experience}
Skills: {my_secrets.skills}
Motivation: {my_target_job.motivation}

State the salary expectations and possible start date as follows:
Salary Expectations: {my_secrets.salary_expectations}
Possible Start Date: {my_secrets.possible_start_date}

Write in a professional, concise, and compact tone.

Sign the cover letter as {my_secrets.name}.
"""

# Get 3 cover letter suggestions
for i in range(3):
    print(f"Cover letter {i+1}")
    print("=" * 5)
    cover_letter_response = get_completion(cover_letter_prompt, temperature=0.7)
    print(cover_letter_response)
    print("=" * 30)